In [49]:
import requests
import datetime as dt
import base64
import time as tm

submit_job_url = "http://dhime.ideam.gov.co/server/rest/services/AtencionCiudadano/DescargarArchivo/GPServer/DescargarArchivo/submitJob"

headers = {
    "Accept": "*/*",
    "Accept-Language": "en-US,en;q=0.9",
    "Content-Type": "application/x-www-form-urlencoded",
    "Origin": "http://dhime.ideam.gov.co",
    "Referer": "http://dhime.ideam.gov.co/atencionciudadano/",
    "Sec-GPC": "1",
}

current_datetime = dt.datetime.now()
previous_date = current_datetime - dt.timedelta(days=1)

field = "TEMPERATURA"
label = "TMX_CON"
day = previous_date.day
month = previous_date.month
year = previous_date.year
data = f"f=json&Filtro=sort%3D%26filter%3D((IdParametro~eq~%27{field}%27~and~Etiqueta~eq~%27{label}%27~and~IdEstacion~eq~%2726205080%27)~or~(IdParametro~eq~%27{field}%27~and~Etiqueta~eq~%27{label}%27~and~IdEstacion~eq~%2727015330%27)~or~(IdParametro~eq~%27{field}%27~and~Etiqueta~eq~%27{label}%27~and~IdEstacion~eq~%2727010810%27))%26group%3D%26fechaInicio%3D{year}-{month}-{day}T05%253A00%253A00.000Z%26fechaFin%3D{year}-{month}-{day}T05%253A00%253A00.000Z%26mostrarGrado%3Dtrue%26mostrarCalificador%3Dtrue%26mostrarNivelAprobacion%3Dtrue&Items=%5B%7B%22IdParametro%22%3A%22{field}%22%2C%22Etiqueta%22%3A%22{label}%22%2C%22EsEjeY1%22%3Afalse%2C%22EsEjeY2%22%3Afalse%2C%22EsTipoLinea%22%3Afalse%2C%22EsTipoBarra%22%3Afalse%2C%22TipoSerie%22%3A%22Estandard%22%2C%22Calculo%22%3A%22%22%7D%2C%7B%22IdParametro%22%3A%22{field}%22%2C%22Etiqueta%22%3A%22{label}%22%2C%22EsEjeY1%22%3Afalse%2C%22EsEjeY2%22%3Afalse%2C%22EsTipoLinea%22%3Afalse%2C%22EsTipoBarra%22%3Afalse%2C%22TipoSerie%22%3A%22Estandard%22%2C%22Calculo%22%3A%22%22%7D%2C%7B%22IdParametro%22%3A%22{field}%22%2C%22Etiqueta%22%3A%22{label}%22%2C%22EsEjeY1%22%3Afalse%2C%22EsEjeY2%22%3Afalse%2C%22EsTipoLinea%22%3Afalse%2C%22EsTipoBarra%22%3Afalse%2C%22TipoSerie%22%3A%22Estandard%22%2C%22Calculo%22%3A%22%22%7D%5D"


response = requests.post(
    f"{submit_job_url}?{data}",
    headers=headers,
    verify=False,
)
response_json = response.json()
job_id = response_json["jobId"]
job_status = response_json["jobStatus"]

print(f"Job ID: {job_id}")
print(f"Job Status: {job_status}")

Job ID: j2e8c37554f674cfc8e5baf7d6b837805
Job Status: esriJobSubmitted


In [55]:
headers = {
    'Accept': '*/*',
    'Accept-Language': 'en-US,en;q=0.9',
    'Content-Type': 'application/x-www-form-urlencoded',
    'Referer': 'http://dhime.ideam.gov.co/atencionciudadano/',
    'Sec-GPC': '1',
}

timestamp = int(tm.time())

response_get_status = requests.get(
    f"http://dhime.ideam.gov.co/server/rest/services/AtencionCiudadano/DescargarArchivo/GPServer/DescargarArchivo/jobs/{job_id}?f=json&dojo.preventCache={timestamp}",
    headers=headers,
    verify=False,
)

response_get_status_json = response_get_status.json()
print(f"Job status: {response_get_status_json['jobStatus']}")

Job status: esriJobSucceeded


In [56]:
headers = {
    "Accept": "*/*",
    "Accept-Language": "en-US,en;q=0.9",
    "Content-Type": "application/x-www-form-urlencoded",
    "Referer": "http://dhime.ideam.gov.co/atencionciudadano/",
    "Sec-GPC": "1",
}

params = {
    "f": "json",
    "returnType": "data",
}

response_get_file = requests.get(
    f"http://dhime.ideam.gov.co/server/rest/services/AtencionCiudadano/DescargarArchivo/GPServer/DescargarArchivo/jobs/{job_id}/results/Archivo",
    params=params,
    headers=headers,
    verify=False,
)

response_json_get_file = response_get_file.json()
file_content = response_json_get_file["value"]

try:
    decoded_file_content = base64.b64decode(file_content)
    with open("datos.zip", "wb") as f:
        f.write(decoded_file_content)
except Exception as e:
    print(str(e))